In [3]:
import itertools
from geopy.distance import geodesic

def travel_time(p1, p2, speed=60):
    d_km = geodesic((p1['Latitude'], p1['Longitude']), (p2['Latitude'], p2['Longitude'])).km
    return (d_km / speed) * 60  # minutes

class OrienteeringOptimizer:
    def __init__(self, points, start, time_limit=480, speed=60):
        self.points = [start] + points  # index 0 is the start
        self.n = len(self.points)
        self.time_limit = time_limit
        self.speed = speed
        self.dist = [[0]*self.n for _ in range(self.n)]
        self.memo = {}

        # Precompute travel times
        for i, a in enumerate(self.points):
            for j, b in enumerate(self.points):
                if i != j:
                    self.dist[i][j] = travel_time(a, b, speed)
    
    def solve(self):
        return self._dp(0, 1, 0)

    def _dp(self, current, visited_mask, time_spent):
        key = (current, visited_mask)
        
        # Check memo only if it's better than previous (prune worse states)
        if key in self.memo and self.memo[key][0] <= time_spent:
            return self.memo[key][1]  # return only result (count, path)

        best_count = bin(visited_mask).count('1') - 1  # exclude start
        best_path = [self.points[current]]

        for next_point in range(1, self.n):  # skip start
            if visited_mask & (1 << next_point):
                continue

            travel = self.dist[current][next_point]
            new_time = time_spent + travel

            if new_time > self.time_limit:
                continue

            new_count, new_path = self._dp(
                next_point,
                visited_mask | (1 << next_point),
                new_time
            )

            if new_count > best_count:
                best_count = new_count
                best_path = [self.points[current]] + new_path

        # Store (time_spent, (count, path))
        self.memo[key] = (time_spent, (best_count, best_path))
        return best_count, best_path


In [ ]:
data_points = [
    {'name': 'Point A', 'Longitude': 3.0588, 'Latitude': 36.7538},
    {'name': 'Point B', 'Longitude': 3.062, 'Latitude': 36.75},
    {'name': 'Point C', 'Longitude': 2.944, 'Latitude': 36.766},
    {'name': 'Point D', 'Longitude': 3.1, 'Latitude': 36.76},
    {'name': 'Point E', 'Longitude': 2.889, 'Latitude': 36.712},
    {'name': 'Point F', 'Longitude': 3.11, 'Latitude': 36.765},
    {'name': 'Point G', 'Longitude': 3.15, 'Latitude': 36.755},
    {'name': 'Point H', 'Longitude': 2.95, 'Latitude': 36.775},
    {'name': 'Point I', 'Longitude': 3.07, 'Latitude': 36.78},
    {'name': 'Point J', 'Longitude': 2.97, 'Latitude': 36.79},
    {'name': 'Point K', 'Longitude': 3.05, 'Latitude': 36.74},
    {'name': 'Point L', 'Longitude': 2.93, 'Latitude': 36.72},
    {'name': 'Point M', 'Longitude': 2.91, 'Latitude': 36.71},
    {'name': 'Point N', 'Longitude': 2.88, 'Latitude': 36.70},
    {'name': 'Point O', 'Longitude': 3.16, 'Latitude': 36.77},
    {'name': 'Point P', 'Longitude': 3.19, 'Latitude': 36.76},
    {'name': 'Point Q', 'Longitude': 3.21, 'Latitude': 36.75},
    {'name': 'Point R', 'Longitude': 3.17, 'Latitude': 36.74},
    {'name': 'Point S', 'Longitude': 2.92, 'Latitude': 36.74},
    {'name': 'Point T', 'Longitude': 2.85, 'Latitude': 36.69}
]
start_point = {'name': 'Start', 'Longitude': 2.9, 'Latitude': 36.75}

optimizer = OrienteeringOptimizer(data_points, start_point, time_limit=480, speed=60)
count, path = optimizer.solve()

print("Optimal path:")
for p in path:
    print(p['name'])

print(f"Points visited: {count}")
